## 生成对抗网络

$$
\min _{G} \max _{D} V(D, G)=\mathbb{E}_{\boldsymbol{x} \sim p_{\text {data }}(\boldsymbol{x})}[\log D(\boldsymbol{x})]+\mathbb{E}_{\boldsymbol{z} \sim p_{\boldsymbol{z}}(\boldsymbol{z})}[\log (1-D(G(\boldsymbol{z})))]
$$

对判别器而言，给定输入数据$\boldsymbol{x}$，期望判别器$D$能够最大化其输出结果，也就是将其预测正确。对于生成器生成的数据$G(z)$而言，是期望最大化$V$, 也就是最大化$-D(G(z))$, 也就是最小化$D(G(z))$, 也就是期望判别器$D$能够将其预测出是假样本。

对于生成器而言，期望生成器能够最小化$V$, 也就是最小化$-D(G(z))$, 也就是最大化$D(G(z))$，也就是期望能够骗过判别器，使得判别器对其输出结果期望是正样本。

## GAN代码实现

### 首先查看一下数据集

In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np

In [2]:
dataset = torchvision.datasets.MNIST("../../data", train=True, download=True,
                                     transform=torchvision.transforms.Compose(
                                         [
                                             torchvision.transforms.Resize(28),
                                             torchvision.transforms.ToTensor(),
                                             # torchvision.transforms.Normalize([0.5], [0.5]),
                                         ]
                                                                             )
                                     )
print(len(dataset))

60000


/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1647804326539/work/torch/csrc/utils/tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
for i in range(len(dataset)):
    if i <= 5:
        print(dataset[i][0].shape, )
    else:
        pass

torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


这里最前面的`1`是通道数。

### 生成器

生成器用于生成图像

In [4]:
image_size = [1, 28, 28]
latent_dim = 96
batch_size = 64
use_gpu = torch.cuda.is_available()

生成器这里都采用的线性层，`torch.prod`是连乘符号。

In [5]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.GELU(),

            nn.Linear(128, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.GELU(),
            nn.Linear(256, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),
            nn.Linear(512, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            nn.Linear(1024, np.prod(image_size, dtype=np.int32)),
            #  nn.Tanh(),
            nn.Sigmoid(),
        )

    def forward(self, z):
        # shape of z: [batchsize, latent_dim]

        output = self.model(z)
        image = output.reshape(z.shape[0], *image_size)

        return image

### 判别器

判别器用于判别图像是否为真实数据。

In [6]:
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(np.prod(image_size, dtype=np.int32), 512),
            torch.nn.GELU(),
            nn.Linear(512, 256),
            torch.nn.GELU(),
            nn.Linear(256, 128),
            torch.nn.GELU(),
            nn.Linear(128, 64),
            torch.nn.GELU(),
            nn.Linear(64, 32),
            torch.nn.GELU(),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, image):
        # shape of image: [batchsize, 1, 28, 28]

        prob = self.model(image.reshape(image.shape[0], -1))

        return prob

训练流程:

In [7]:

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

generator = Generator()
discriminator = Discriminator()


g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0003, betas=(0.4, 0.8), weight_decay=0.0001)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0003, betas=(0.4, 0.8), weight_decay=0.0001)

loss_fn = nn.BCELoss()
labels_one = torch.ones(batch_size, 1)
labels_zero = torch.zeros(batch_size, 1)

if use_gpu:
    print("use gpu for training")
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    loss_fn = loss_fn.cuda()
    labels_one = labels_one.to("cuda")
    labels_zero = labels_zero.to("cuda")

num_epoch = 20
for epoch in range(num_epoch):
    for i, mini_batch in enumerate(dataloader):
        gt_images, _ = mini_batch


        z = torch.randn(batch_size, latent_dim)

        if use_gpu:
            gt_images = gt_images.to("cuda")
            z = z.to("cuda")

        pred_images = generator(z)
        g_optimizer.zero_grad()

        recons_loss = torch.abs(pred_images-gt_images).mean()

        g_loss = recons_loss*0.05 + loss_fn(discriminator(pred_images), labels_one)

        g_loss.backward()
        g_optimizer.step()

        d_optimizer.zero_grad()

        real_loss = loss_fn(discriminator(gt_images), labels_one)
        fake_loss = loss_fn(discriminator(pred_images.detach()), labels_zero)
        d_loss = (real_loss + fake_loss)

        # 观察real_loss与fake_loss，同时下降同时达到最小值，并且差不多大，说明D已经稳定了

        d_loss.backward()
        d_optimizer.step()

        if i % 50 == 0:
            print(f"step:{len(dataloader)*epoch+i}, recons_loss:{recons_loss.item()}, g_loss:{g_loss.item()}, d_loss:{d_loss.item()}, real_loss:{real_loss.item()}, fake_loss:{fake_loss.item()}")

        if i % 400 == 0:
            image = pred_images[:16].data
            
            torchvision.utils.save_image(image, f"./image_gan_result/image_{len(dataloader)*epoch+i}.png", nrow=4)



step:0, recons_loss:0.4742576479911804, g_loss:0.7776899933815002, d_loss:1.3897684812545776, real_loss:0.7539621591567993, fake_loss:0.6358063220977783
step:50, recons_loss:0.14779216051101685, g_loss:0.5809261798858643, d_loss:1.5458400249481201, real_loss:0.6446172595024109, fake_loss:0.9012227654457092
step:100, recons_loss:0.16047462821006775, g_loss:0.9785372018814087, d_loss:1.4482042789459229, real_loss:0.9713031053543091, fake_loss:0.47690120339393616
step:150, recons_loss:0.15550920367240906, g_loss:0.5756874084472656, d_loss:1.3391588926315308, real_loss:0.44106772541999817, fake_loss:0.8980911374092102
step:200, recons_loss:0.15602752566337585, g_loss:0.6427386403083801, d_loss:1.3159592151641846, real_loss:0.55678790807724, fake_loss:0.7591712474822998
step:250, recons_loss:0.156376913189888, g_loss:0.7382826209068298, d_loss:1.3877520561218262, real_loss:0.7297801375389099, fake_loss:0.6579718589782715
step:300, recons_loss:0.1615169495344162, g_loss:0.9401568174362183, d

step:2624, recons_loss:0.1615467369556427, g_loss:1.420257329940796, d_loss:1.2366915941238403, real_loss:0.9369018077850342, fake_loss:0.29978981614112854
step:2674, recons_loss:0.15933969616889954, g_loss:0.8582610487937927, d_loss:1.1580445766448975, real_loss:0.5865167379379272, fake_loss:0.571527898311615
step:2724, recons_loss:0.16400876641273499, g_loss:0.9662281274795532, d_loss:1.1819748878479004, real_loss:0.6647489666938782, fake_loss:0.517225980758667
step:2774, recons_loss:0.1642177402973175, g_loss:0.7683197259902954, d_loss:1.181639313697815, real_loss:0.4560427963733673, fake_loss:0.72559654712677
step:2811, recons_loss:0.16236163675785065, g_loss:0.8205521702766418, d_loss:1.170367956161499, real_loss:0.5471788644790649, fake_loss:0.6231891512870789
step:2861, recons_loss:0.16325780749320984, g_loss:0.7032178044319153, d_loss:1.2196314334869385, real_loss:0.47801849246025085, fake_loss:0.7416129112243652
step:2911, recons_loss:0.16517670452594757, g_loss:0.764766454696

step:5235, recons_loss:0.1729467213153839, g_loss:1.473178505897522, d_loss:1.207217812538147, real_loss:0.9241201877593994, fake_loss:0.28309765458106995
step:5285, recons_loss:0.16946087777614594, g_loss:0.7867621183395386, d_loss:1.1053816080093384, real_loss:0.41556990146636963, fake_loss:0.6898117065429688
step:5335, recons_loss:0.16575472056865692, g_loss:0.9450920820236206, d_loss:1.107523798942566, real_loss:0.5332910418510437, fake_loss:0.5742327570915222
step:5385, recons_loss:0.1596365123987198, g_loss:1.244154691696167, d_loss:1.0869392156600952, real_loss:0.67775958776474, fake_loss:0.4091796576976776
step:5435, recons_loss:0.17131498456001282, g_loss:1.2587268352508545, d_loss:0.7526278495788574, real_loss:0.36539456248283386, fake_loss:0.38723331689834595
step:5485, recons_loss:0.15822894871234894, g_loss:1.3994280099868774, d_loss:1.0779963731765747, real_loss:0.7720855474472046, fake_loss:0.3059108555316925
step:5535, recons_loss:0.1618402898311615, g_loss:0.9120813012

step:7846, recons_loss:0.1598212569952011, g_loss:1.3285719156265259, d_loss:0.7920563220977783, real_loss:0.4383239150047302, fake_loss:0.3537324070930481
step:7896, recons_loss:0.1621427983045578, g_loss:0.8291987776756287, d_loss:1.0287789106369019, real_loss:0.33305448293685913, fake_loss:0.6957244277000427
step:7946, recons_loss:0.16873987019062042, g_loss:1.3078981637954712, d_loss:1.0541876554489136, real_loss:0.7263352274894714, fake_loss:0.32785242795944214
step:7996, recons_loss:0.16579113900661469, g_loss:0.7241503596305847, d_loss:1.200965404510498, real_loss:0.29968440532684326, fake_loss:0.90128093957901
step:8046, recons_loss:0.17134390771389008, g_loss:1.030924916267395, d_loss:0.9909774661064148, real_loss:0.5073801279067993, fake_loss:0.4835973381996155
step:8096, recons_loss:0.16601704061031342, g_loss:0.8861343264579773, d_loss:1.0815527439117432, real_loss:0.4840579032897949, fake_loss:0.597494900226593
step:8146, recons_loss:0.1643712967634201, g_loss:1.1985527276

step:10457, recons_loss:0.15859878063201904, g_loss:1.192583441734314, d_loss:0.9421073198318481, real_loss:0.5171555280685425, fake_loss:0.42495179176330566
step:10507, recons_loss:0.18841160833835602, g_loss:0.8863096833229065, d_loss:1.0333335399627686, real_loss:0.42016729712486267, fake_loss:0.6131662130355835
step:10557, recons_loss:0.162751704454422, g_loss:1.0534770488739014, d_loss:0.9891139268875122, real_loss:0.4834766983985901, fake_loss:0.5056372284889221
step:10607, recons_loss:0.16870708763599396, g_loss:1.2008370161056519, d_loss:1.0937970876693726, real_loss:0.5868080854415894, fake_loss:0.5069890022277832
step:10657, recons_loss:0.16280102729797363, g_loss:1.8319566249847412, d_loss:0.6154624223709106, real_loss:0.42839914560317993, fake_loss:0.1870633065700531
step:10707, recons_loss:0.15973123908042908, g_loss:1.4514861106872559, d_loss:0.8885369300842285, real_loss:0.5983750224113464, fake_loss:0.2901618778705597
step:10757, recons_loss:0.16521765291690826, g_loss:

...

step:175419, recons_loss:0.1566716730594635, g_loss:1.1347254514694214, d_loss:1.0791988372802734, real_loss:0.46831655502319336, fake_loss:0.6108822822570801
step:175469, recons_loss:0.16089095175266266, g_loss:1.422531247138977, d_loss:0.8602066040039062, real_loss:0.4321407079696655, fake_loss:0.4280658960342407
step:175519, recons_loss:0.16278718411922455, g_loss:1.4676423072814941, d_loss:0.8924481868743896, real_loss:0.4958566129207611, fake_loss:0.39659157395362854
step:175569, recons_loss:0.16051410138607025, g_loss:1.1966551542282104, d_loss:0.8729305267333984, real_loss:0.3938131332397461, fake_loss:0.47911739349365234
step:175619, recons_loss:0.16056077182292938, g_loss:1.4777804613113403, d_loss:0.8857380151748657, real_loss:0.5309708714485168, fake_loss:0.35476717352867126
step:175669, recons_loss:0.16084399819374084, g_loss:1.803045630455017, d_loss:0.8211629986763, real_loss:0.5069212317466736, fake_loss:0.31424176692962646
step:175719, recons_loss:0.16014941036701202, g

step:177993, recons_loss:0.15805166959762573, g_loss:1.4171407222747803, d_loss:0.9520875811576843, real_loss:0.4321795105934143, fake_loss:0.51990807056427
step:178030, recons_loss:0.16259436309337616, g_loss:1.1403933763504028, d_loss:0.8753401041030884, real_loss:0.353192538022995, fake_loss:0.522147536277771
step:178080, recons_loss:0.16702581942081451, g_loss:1.4474350214004517, d_loss:1.0553889274597168, real_loss:0.5484575629234314, fake_loss:0.5069313645362854
step:178130, recons_loss:0.15856413543224335, g_loss:1.7398452758789062, d_loss:0.7735071182250977, real_loss:0.4119650721549988, fake_loss:0.3615420162677765
step:178180, recons_loss:0.16343039274215698, g_loss:1.6207146644592285, d_loss:1.0103224515914917, real_loss:0.5862777233123779, fake_loss:0.42404472827911377
step:178230, recons_loss:0.164120614528656, g_loss:1.1380842924118042, d_loss:0.9563071727752686, real_loss:0.4075506329536438, fake_loss:0.5487565398216248
step:178280, recons_loss:0.15809547901153564, g_los

step:180554, recons_loss:0.1604612022638321, g_loss:1.4496973752975464, d_loss:1.0292357206344604, real_loss:0.6269997358322144, fake_loss:0.4022359848022461
step:180604, recons_loss:0.16936898231506348, g_loss:1.4341204166412354, d_loss:0.9518386721611023, real_loss:0.5709022879600525, fake_loss:0.3809363842010498
step:180654, recons_loss:0.16809821128845215, g_loss:1.3596973419189453, d_loss:0.886097252368927, real_loss:0.3968701958656311, fake_loss:0.4892270565032959
step:180704, recons_loss:0.16093489527702332, g_loss:1.4206904172897339, d_loss:0.8876955509185791, real_loss:0.431385338306427, fake_loss:0.4563101828098297
step:180754, recons_loss:0.1639549732208252, g_loss:1.1630215644836426, d_loss:1.0893845558166504, real_loss:0.5151201486587524, fake_loss:0.574264407157898
step:180804, recons_loss:0.1651015728712082, g_loss:1.6446279287338257, d_loss:0.9820499420166016, real_loss:0.5613794922828674, fake_loss:0.42067044973373413
step:180841, recons_loss:0.16127583384513855, g_los

step:183115, recons_loss:0.16061459481716156, g_loss:1.2975188493728638, d_loss:0.9444237947463989, real_loss:0.42516523599624634, fake_loss:0.5192585587501526
step:183165, recons_loss:0.15825419127941132, g_loss:1.6993898153305054, d_loss:0.8991144299507141, real_loss:0.5593675971031189, fake_loss:0.3397468328475952
step:183215, recons_loss:0.1649269163608551, g_loss:1.7883728742599487, d_loss:0.927297830581665, real_loss:0.5810995697975159, fake_loss:0.3461982309818268
step:183265, recons_loss:0.16677270829677582, g_loss:1.4735121726989746, d_loss:0.9383307695388794, real_loss:0.521943986415863, fake_loss:0.41638681292533875
step:183315, recons_loss:0.1663307398557663, g_loss:1.1241588592529297, d_loss:0.8536466360092163, real_loss:0.33661046624183655, fake_loss:0.5170361399650574
step:183365, recons_loss:0.1597154587507248, g_loss:1.2739381790161133, d_loss:0.9938059449195862, real_loss:0.4940807819366455, fake_loss:0.4997251629829407
step:183415, recons_loss:0.16114872694015503, g_

step:185676, recons_loss:0.16511337459087372, g_loss:1.3504339456558228, d_loss:0.9877408742904663, real_loss:0.5535268187522888, fake_loss:0.4342140853404999
step:185726, recons_loss:0.17141276597976685, g_loss:1.3382291793823242, d_loss:0.903479814529419, real_loss:0.3873341381549835, fake_loss:0.5161457061767578
step:185776, recons_loss:0.155216783285141, g_loss:1.6975176334381104, d_loss:0.8588893413543701, real_loss:0.4889751970767975, fake_loss:0.36991411447525024
step:185826, recons_loss:0.16758431494235992, g_loss:1.2451273202896118, d_loss:0.7985703349113464, real_loss:0.3451196551322937, fake_loss:0.45345067977905273
step:185876, recons_loss:0.16281825304031372, g_loss:1.3802893161773682, d_loss:0.8296172618865967, real_loss:0.42674386501312256, fake_loss:0.4028733968734741
step:185926, recons_loss:0.1636272519826889, g_loss:1.7969516515731812, d_loss:0.9069026708602905, real_loss:0.595762312412262, fake_loss:0.31114038825035095
step:185976, recons_loss:0.16236817836761475, g

## 训练总结


1）引入batchnorm可以提高收敛速度，具体做法是在生成器的Linear层后面添加BatchNorm1d，最后一层除外，判别器不要加
（2）直接预测【0,1】之间的像素值即可，不做归一化的transform；或者也可以放大，预测【-1,1】之间，用mean=0.5 std=0.5进行归一化transform都可以
（3）将激活函数ReLU换成GELU效果更好
（4）real_loss基于真实图片，fake_loss基于生成图片，real_loss = loss_fn(discriminator(gt_images), torch.ones(batch_size, 1))，fake_loss = loss_fn(discriminator(pred_images.detach()), torch.zeros(batch_size, 1))
（5）适当引入重构loss，计算像素值的L1误差
（6）建议引入loss打印语句，如：print(f"step:{len(dataloader)*epoch+i}, g_loss:{g_loss.item()}, d_loss:{d_loss.item()}, real_loss:{real_loss.item()}, fake_loss:{fake_loss.item()}")
（7）判别器模型容量不宜过大
（8）save_image中的normalize设置成True，目的是将像素值min-max自动归一到【0,1】范围内，如果已经预测了【0,1】之间，则可以不用设置True
（9）判别器的学习率不能太小
（10）Adam的一阶平滑系数和二阶平滑系数 betas 适当调小一点，可以帮助学习，设置一定比例的weight decay